In [32]:
from pyspark.sql.functions import *
from pyspark.sql import functions as F
import pandas as pd

In [33]:
predictions = pd.read_parquet('../data/curated/transaction_predictions.parquet')
merchants_df = pd.read_parquet('../data/curated/merchants.parquet')

In [11]:
merchants_df = merchants_df[['merchant_abn', 'name','segment']]
total_ranking = predictions.groupby('merchant_abn').sum().sort_values(by = 'value', ascending = False)
total_ranking = total_ranking[['value']]
segments_ranking = merchants_df.join(total_ranking, on='merchant_abn', how='right')
total_ranking = segments_ranking[['merchant_abn','name','value']]

misc_df = segments_ranking[segments_ranking['segment'] == 'Miscellaneous']
misc_df = misc_df.sort_values(by = 'value', ascending = False)

home_df = segments_ranking[segments_ranking['segment'] == 'Home']
home_df = home_df.sort_values(by = 'value', ascending = False)

acc_df = segments_ranking[segments_ranking['segment'] == 'Accessories']
acc_df = acc_df.sort_values(by = 'value', ascending = False)

art_df = segments_ranking[segments_ranking['segment'] == 'Art']
art_df = art_df.sort_values(by = 'value', ascending = False)

tech_df = segments_ranking[segments_ranking['segment'] == 'Media & Technology']
tech_df = tech_df.sort_values(by = 'value', ascending = False)

In [13]:
total_ranking.sort_values(by = 'value', ascending = True).head(20)

,merchant_abn,name,value
2060,55403018592,Elit Limited,5.773170e-15
1851,50532670634,Accumsan Laoreet Ipsum Company,5.773170e-15
2640,68591542501,Felis Ltd,5.773170e-15
2,10165489824,Nunc Sed Company,5.773170e-15
594,22853038342,Semper Pretium Limited,5.773170e-15
3428,86201937910,Libero Et LLC,5.773170e-15
3184,80426072728,Massa Non Ante Industries,5.773170e-15
3029,76866488151,Euismod Urna Company,5.773170e-15
381,18261886835,Massa Rutrum LLP,5.773170e-15
2067,55555661470,Nullam Scelerisque LLC,5.773170e-15


In [3]:
from pyspark.sql import SparkSession


spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/10/09 17:22:45 WARN Utils: Your hostname, dash_surface resolves to a loopback address: 127.0.1.1; using 172.24.198.246 instead (on interface eth0)
22/10/09 17:22:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 17:22:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [35]:
predictions = spark.read.parquet('../data/curated/transaction_predictions.parquet')

In [36]:
predictions

merchant_idx,week_idx,merchant_abn,value
0,86,10023283211,0.5608883
0,87,10023283211,0.5671433
0,88,10023283211,0.5668918
0,89,10023283211,0.589157
0,90,10023283211,0.5888265
0,91,10023283211,0.59795564
0,92,10023283211,0.5868635
0,93,10023283211,0.5869298
0,94,10023283211,0.60623986
0,95,10023283211,0.610332


In [4]:
transactions_sdf = spark.read.parquet('../data/curated/cleaned_transactions.parquet/')

In [15]:
# Number 1 ranked merchant

topMerchant = transactions_sdf.where(F.col('merchant_abn') == 79827781481)

In [16]:
total = topMerchant.count()
maleCount = topMerchant.where(F.col('gender') == "Male").count()
femaleCount = topMerchant.where(F.col('gender') == "Female").count()
undisCount = topMerchant.where(F.col('gender') == "Undisclosed").count()

In [19]:
print(f"Top merchant male proportion: {maleCount/total}")
print(f"Top merchant female proportion: {femaleCount/total}")
print(f"Top merchant undisclosed proportion: {undisCount/total}")
print(f"Top merchant total count: {total}")

Top merchant male proportion: 0.45229681978798586
Top merchant female proportion: 0.44617196702002354
Top merchant undisclosed proportion: 0.10153121319199057
Top merchant total count: 4245


In [28]:
# Worst ranked merchant

badMerchant = transactions_sdf.where(F.col('merchant_abn') == 55403018592)

In [29]:
btotal = badMerchant.count()
bmaleCount = badMerchant.where(F.col('gender') == "Male").count()
bfemaleCount = badMerchant.where(F.col('gender') == "Female").count()
bundisCount = badMerchant.where(F.col('gender') == "Undisclosed").count()

In [31]:
print(f"Worst merchant male proportion: {bmaleCount/btotal}")
print(f"Worst merchant female proportion: {bfemaleCount/btotal}")
print(f"Worst merchant undisclosed proportion: {bundisCount/btotal}")
print(f"Worst merchant total count: {btotal}")

Worst merchant male proportion: 1.0
Worst merchant female proportion: 0.0
Worst merchant undisclosed proportion: 0.0
Worst merchant total count: 1
